In [1]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter



institutions_df = pd.read_excel('out\polon_discp.xlsx')

In [2]:
institutions_df.columns

Index(['uid', 'name', 'status', 'fullName', 'code',
       'supervisingInstitutionName', 'kind', 'dateFrom', 'regon', 'www',
       'phoneNumber', 'email', 'isLaboratory', 'isLibrary', 'isScientific',
       'country', 'city', 'street', 'buildingNumber', 'postalCode',
       'postalCity', 'voivodeship', 'name_clean', 'liquidationStart',
       'faxNumber', 'localNumber', 'dateTo', 'nip', 'universityType',
       'scientificInstitutionType', 'krs', 'Unikalny identyfikator rekordu',
       'Nazwa instytucji', 'Okres ewaluacji', 'dos_archeologia',
       'dos_architektura i urbanistyka', 'dos_astronomia',
       'dos_automatyka, elektronika i elektrotechnika',
       'dos_ekonomia i finanse', 'dos_filozofia',
       'dos_geografia społeczno-ekonomiczna i gospodarka przestrzenna',
       'dos_historia', 'dos_informatyka',
       'dos_informatyka techniczna i telekomunikacja',
       'dos_inżynieria biomedyczna', 'dos_inżynieria chemiczna',
       'dos_inżynieria lądowa i transport', 'dos_i

In [3]:

str_none = institutions_df['street'].isna()

institutions_df.loc[~str_none, 'address'] = institutions_df[~str_none]['street'].apply(lambda st: str(st).replace('ul.','').replace('al.','').replace('pl.','').strip()) +' ' + institutions_df['buildingNumber'] + ', ' + institutions_df['postalCode'] + ', ' + institutions_df['city'] + ', ' + institutions_df['country']
institutions_df.loc[str_none, 'address'] = institutions_df['postalCode'] + ', ' + institutions_df['city'] +' ' + institutions_df['buildingNumber']+ ', ' + institutions_df['country']

In [4]:

locator = Nominatim(user_agent="myGeocoder")

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
institutions_df['location'] = institutions_df['address'].apply(geocode)
institutions_df['point'] = institutions_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

In [5]:
na_point = institutions_df['point'].isna()

institutions_df['address2'] =  institutions_df['city'] + ', ' + institutions_df['postalCode'] +', ' + institutions_df['country']

institutions_df.loc[na_point,'location'] = institutions_df.loc[na_point,'address2'].apply(geocode)
institutions_df.loc[na_point,'point'] = institutions_df.loc[na_point,'location'].apply(lambda loc: tuple(loc.point) if loc else None)



In [6]:
na_point = institutions_df['point'].isna()

institutions_df['address3'] =  institutions_df['city'] + ', ' +institutions_df['country']

institutions_df.loc[na_point,'location'] = institutions_df.loc[na_point,'address3'].apply(geocode)
institutions_df.loc[na_point,'point'] = institutions_df.loc[na_point,'location'].apply(lambda loc: tuple(loc.point) if loc else None)



In [7]:
del institutions_df['address']
del institutions_df['address2']
del institutions_df['address3']

In [8]:

institutions_df.to_excel('out/polon_discp.xlsx', index=False, encoding='utf-8')
